#**Space-based Tokenization**

In [65]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/clean_shakespeare.txt'
file_path_friends =


SyntaxError: invalid syntax (2054320065.py, line 5)

In [1]:
with open (r"Shakespeare_clean_train.txt", 'r') as f:
  text_train = f.read()
with open (r"Shakespeare_clean_valid.txt", 'r') as f:
  text_valid = f.read()
with open (r"Shakespeare_clean_test.txt", 'r') as f:
  text_test = f.read()

In [2]:
import nltk
from collections import Counter



def get_words(text):

  pattern = r'''(?x)          # set flag to allow verbose regexps
          (?:[A-Z]\.)+        # abbreviations, e.g. U.S.A.
          | \w+'\w+           # contractions
        | \w+(?:-\w+)*        # words with optional internal hyphens
        | \$?\d+(?:\.\d+)?%?  # currency and percentages, e.g. $12.40, 82%
        | \.\.\.              # ellipsis
        | [][.,;"'?():_`-]    # these are separate tokens; includes ], [
      '''

  vocab = nltk.regexp_tokenize(text, pattern)
  vocab = [word.lower() for word in vocab]
  unique_words = set(vocab)
  c = Counter(vocab)

  sorted_dict = {key: value for key, value in sorted(
      c.items(), key=lambda item: item[1], reverse=True)}

  return sorted_dict

# Byte Pair Encoding

- make Shakespeare into test 1% and train
- train the segmenter with varying k (try normalization strategies)
- compare the performance against a different set
- figure out a measure for accuracy

In [3]:
from itertools import islice

def performance(sorted_dict, vocab, k=10000):

    def take(n, iterable):
        """Return the first n items of the iterable as a list."""
        return list(islice(iterable, n))


    # extract top k keys
    qualifier = [key for key, _ in take(k, sorted_dict.items())]

    # clean vocab by stripping the _
    cleaned_vocab = {word.rstrip('_') for word in vocab}

    # calculate overlap
    overlap = set(qualifier) & cleaned_vocab
    num_overlap = len(overlap)

    # percentage
    percentage = (num_overlap / len(qualifier)) * 100
    return percentage


In [4]:
import numpy as np
from collections import defaultdict

from collections import OrderedDict
from collections import Counter


def bpe(dictionary, k=None):
  """
  Input:
    dictionary (dict): a dictionary that contains the tokens and their respective counts
  return:
    vocab_bpe (list): vocabulary of the corpus
    sorted_token_freq:
    dict_matrix:
  """

  # get all unique characters in original corpus
  all_keys = "_ ".join(dictionary.keys())
  vocab_bpe = list(set(all_keys))

  # Corpus/dictionary in einzelne tokens splitten, nach jedem Wort (VOR space!) "_" einfügen
    # worte als list of characters
  dict_matrix = []
  for key, value in dictionary.items():
    new_key = list(f"{str(key)}_ ")
    dict_matrix.append([new_key, value])


  # NICHT corpus, sondern liste an Wörtern in einzelne tokens splitten,
  # >> jede occurence mit counts der Worte multiplizieren


  token_freq = defaultdict(int) # TODO: does that need to be moved outside of the loop?
  iteration = True
  num_rounds = 0
  while iteration:

    for token_list, value in dict_matrix:
      for i in range(len(token_list)-2):
        # wollen den und den nächsten token als key
        search_key = token_list[i] + token_list[i+1]
        # zu dictionary hinzufügen falls key noch nicht existiert
        token_freq[search_key] += value
    # word_freqs: gehen jede existierende Folge aus zwei tokens in list of words von vorne bis hinten durch

    c = Counter(token_freq)

    sorted_token_freq = {key: value for key, value in sorted(
        c.items(), key=lambda item: item[1], reverse=True)}

    # Find the most frequent token not already in vocab_bpe
    for token in sorted_token_freq.keys():
      if token not in vocab_bpe:
        first_token = token
        break
      else:
        first_token = None  # Optional: fallback in case all tokens are already in vocab

    if first_token:
      vocab_bpe.append(first_token)
    else:
      print("No new token to add.")

    # höchster count wird gemerged:
    # add to vocab
    #first_token = list(sorted_token_freq.keys())[0]
    #vocab_bpe.append(first_token)
    # replace in list of words
    # start again?
    for i in range(len(dict_matrix)):
        token_list, value = dict_matrix[i]
        j = 0
        while j < len(token_list) - 1:
            search_key = token_list[j] + token_list[j + 1]
            if search_key == first_token:
                merged_token = search_key
                # Merge the tokens
                token_list = token_list[:j] + [merged_token] + token_list[j + 2:]
                # Don't increment j — might be able to merge again
            else:
                j += 1
        dict_matrix[i][0] = token_list
    if k:
       k -= 1
       iteration = (k > 0)

    else:
       num_rounds += 1
       accuracy = performance(dictionary, vocab_bpe, 500)
       iteration != (accuracy > 70)

       if num_rounds > 1500:
          print("exceeded, accuracy: ", accuracy)


          iteration = False

  return vocab_bpe, sorted_token_freq, dict_matrix

In [5]:
dict_train = get_words(text_train)
dict_test = get_words(text_test)

vocab_train, sorted_token_freq_train, dict_matrix_train = bpe(dict_train)

exceeded, accuracy:  76.8


In [6]:
print(f"Shakespeare vocabulary: {vocab_train}")
print(f"Shakespeare token_freq: {sorted_token_freq_train}")
print(f"Shakespeare dict_matrix: {dict_matrix_train}")

Shakespeare vocabulary: ['l', 'w', 'v', 'c', '8', ' ', '9', 'i', 'b', ':', 'n', 'r', 'e', 'm', 'o', 'a', 'z', 'h', '.', 't', '1', 'j', "'", '-', 'u', 's', 'y', 'k', '2', '6', 'g', 'q', '_', '?', 'd', 'f', 'x', 'p', ',', ';', 'e_', 'th', 's_', 't_', ',_', 'd_', 'r_', 'an', 'y_', 'n_', 'o_', 'ou', 'er', 'in', '._', 'en', 'ha', 'l_', 'ar', 'or', 'll', 'on', 'the_', 'ea', 'f_', 'no', 'a_', 'is_', 'i_', 'and_', 'he', 'es', 'the', 'er_', 'hi', 'wi', 'om', 're', 'le', 'th_', ';_', 'm_', 'me_', 'ti', 'you', ':_', 'to_', 'g_', 'st', 'al', 'll_', 'at_', 'el', 'lo', 've_', 'ow', 'ma', 'h_', 'in_', 'us_', 'at', 'ch', 'es_', 'li', 'k_', 'se', 'st_', 'ro', 'ed_', 'it_', 'ri', '?_', 'of_', 'la', 'or_', 'w_', 'on_', 'my_', 'ca', 'ra', 'sh', 'ou_', 'be', 'wh', 'ur', 'si', 'ee', 'ce_', 'an_', 'te', 'as_', 'se_', 'de', 'oo', 'he_', 'bu', 'di', 'gh', 'fo', 'is', 'ld_', 'ay_', 'you_', 'en_', 'as', 'not_', 'un', 'to', 'tr', 'ing_', "'s_", "'_", 'me', 'ere_', 'sa', 'ol', 'be_', 'ta', 'et_', 'ke_', 'that_', '

In [7]:
train_accuracy = performance(dict_train, vocab_train, 500)
test_accuracy = performance(dict_test, vocab_train, 500)

print("train accuracy: ", train_accuracy)
print("test accuracy: ", test_accuracy)

train accuracy:  76.8
test accuracy:  66.8


In [8]:
import tiktoken

def compare_to_gpt_encoding(text, alphabet, model_name="gpt-3.5-turbo"):
    """
    Encodes a text string using tiktoken and prints the tokens along with their IDs.

    Args:
        text: The text string to encode.

    """
    alphabet = {word.rstrip('_') for word in alphabet}
    try:
        encoding = tiktoken.encoding_for_model(model_name)
    except KeyError:
        print(f"Warning: Model '{model_name}' not found.  Using 'cl100k_base' instead.")
        encoding = tiktoken.get_encoding("cl100k_base")

    tokens = encoding.encode(text)
    tp = 0

    unique_tokens = set(tokens)
    for token in unique_tokens:
      decoded_token = encoding.decode([token])
      #check if the decoded token is in the dict alphabet
      if(decoded_token in alphabet):
        tp = tp+1
    total_len = len(alphabet)
    print(tp/total_len)


with open (file_path, 'r') as f:
  text = f.read()
model_name="gpt-3.5-turbo"
tokens_train = compare_to_gpt_encoding(text_train, vocab_train, model_name)
tokens_test = compare_to_gpt_encoding(text_test, vocab_train, model_name)

NameError: name 'file_path' is not defined

# Different Corpus: Friends TV Show

In [ ]:
with open ('friends.txt', 'r') as f:
  friends = f.read()

friends_dict = get_words(friends)
friends_accuracy = performance(friends_dict, vocab_train, 500)
print("Accuracy on Friends: ", friends_accuracy)

tokens_friends = compare_to_gpt_encoding(friends, vocab_train, model_name)

FileNotFoundError: [Errno 2] No such file or directory: 'friends.txt'

# Task 2

## N-Grams (first uni-gram, gradual increasing to 4, have n as input, Laplace-Smoothing)
## Perplexity

## Interpolation vs Backoff

## Generator

In [9]:
from generator import to_byte_pair

In [141]:
from collections import Counter, defaultdict

class N_gram:

  def __init__(self, corpus, n):
    self.ndim = n
    self.unigram_probs = self.get_unigram_probs(corpus)
    self.split_text = self.split_ngrams(corpus, n)
    self.n_gram_probs = self.calculate_n_gram_probs(self.split_text, n)



  def get_unigram_probs(self, corpus):
    """
    Pass corpus (already to byte-pair) and get unigram probs.
    """
    c = Counter(corpus)
    total = sum(c.values())
    unigram_probs = {token: count / total for token, count in c.items()} # is that normalised the way we want it
    return unigram_probs


  def split_ngrams(self, corpus, n):
    """
    Pass corpus (already to byte-pair)
    Funtion that chunks corpus into n-grams (n tokens are chunked together)
    """
    # want to split this into the maximum possible length
    # why am I not passing the vocabulary?
    print(type(corpus))
    split_text = []
    for i in range(len(corpus) - n + 1):
      
      split_text.append(corpus[i : i+n])

    return split_text


  def calculate_n_gram_probs(self, split_text, n):
    """
    Builds conditional probabilities: P(w_n | w_1, ..., w_{n-1})
    Currently uses nested dictionaries, need to make that more useful for you
    """
    if n == 1:
      return self.unigram_probs
    
    # nested defaultdicts for automatic initialization
    n_gram_counts = defaultdict(lambda: defaultdict(int))

    for n_gram in split_text:
      prefix = tuple(n_gram[:-1])  # context: first n-1 tokens
      target = n_gram[-1]          # prediction target: nth token
      n_gram_counts[prefix][target] += 1

    # convert counts to probabilities
    n_gram_probs = {}

    for prefix, target_counts in n_gram_counts.items():
      total = float(sum(target_counts.values()))
      n_gram_probs[prefix] = {token: count / total for token, count in target_counts.items()}


    return n_gram_probs

  def perplexity(self, split_text, n_gram_probs):
    """
    Calculate perplexity of a given text based on an n-gram model.

    Args:
        split_text (str): Input text string, use test text.
        n_gram_probs (dict of dict): ...
    Returns:
        float: Perplexity value of the input text.
    """

    #text = generator.to_byte_pair.to_byte_pair(text,vocab) # already done
    #n = self.ndim  # order of the n-gram model # can access through class
    prob_total = 1.0

    # Iterate through the text, considering n-grams
    for t in range(len(split_text) - self.ndim):
      context = tuple(split_text[t:t + self.ndim - 1])  # (n-1)-gram
      next_token = split_text[t + self.ndim - 1]        # next token after context

      # Assign very small probability for unknown n-grams (smoothing) otherwise retrieve actual probability
      prob = 0.0
      if context in n_gram_probs and next_token in n_gram_probs[context]:
        prob = n_gram_probs[context][next_token]
      else:
        prob = 1e-10  # unknown n-gram

      prob_total *= prob

    print(f"Total probability: {prob_total}")
    if prob_total > 0.0:
      perplexity_value = prob_total ** (-1 / len(split_text))
    else:
      perplexity_value = float('inf')

    print(f"Perplexity value: {perplexity_value}")
    return perplexity_value


  def intrinsic_eval(self, corpus_test, n_gram_probs):
    perplexity_value = self.perplexity(corpus_test, n_gram_probs)


In [123]:
liste = ['a' , 1, 2, 4]
print(liste[:0])

[]


In [12]:
# make corpus to byte_pairs
n_gram_corps_train = to_byte_pair(text_train, vocab_train)

# idea would be to pass n_gram_corps to the unigram function

In [13]:
Shakespeare_byte = open('Shakespeare_byte.txt', 'x')
Shakespeare_byte.write(str(n_gram_corps_train))
Shakespeare_byte.close()

In [14]:
n_gram_corps_test = to_byte_pair(text_test, vocab_train)
#print(n_gram_corps_test[:25])

In [15]:
Shakespeare_byte_test = open('Shakespeare_byte_test.txt', 'x')
Shakespeare_byte_test.write(str(n_gram_corps_test))
Shakespeare_byte_test.close()

In [16]:
n_gram_corps_valid = to_byte_pair(text_valid, vocab_train)

Shakespeare_byte_valid = open('Shakespeare_byte_valid.txt', 'x')
Shakespeare_byte_valid.write(str(n_gram_corps_valid))
Shakespeare_byte_valid.close()

In [13]:
with open('Shakespeare_byte.txt', 'r') as f:
    n_gram_corps_train = f.read()

In [15]:
with open('Shakespeare_byte_test.txt', 'r') as f:
    n_gram_corps_test = f.read()

In [142]:
my_unigram = N_gram(n_gram_corps_train, 1)
my_bigram = N_gram(n_gram_corps_train, 2)
#print(f"Unigram probabilities: {my_unigram.unigram_probs.items()}")
print(f"Unigram split: {my_unigram.split_text[:10]}")
print(f"Bigram split: {my_bigram.split_text[:10]}")
my_bigram_probs = my_bigram.calculate_n_gram_probs(my_bigram.split_text, 2)
my_bigram_perplexity = my_bigram.perplexity(n_gram_corps_test, my_bigram_probs)

#print(f"Bigram probabilities: {my_bigram_probs}")

<class 'str'>
<class 'str'>
Unigram split: ['H', 'a', 'l', 'l', ' ', 'i', 's', 't', ' ', 'b']
Bigram split: ['Ha', 'al', 'll', 'l ', ' i', 'is', 'st', 't ', ' b', 'bl']
Total probability: 0.0
Perplexity value: inf


In [138]:
print(my_unigram.n_gram_probs['cleopatra'])

KeyError: 'cleopatra'

In [126]:
import operator
import random

def generate(context, ngrams, n, vocab):
    """
    Generate text using an n-gram given a defined context.

    Args:
        context (str): The initial context string.
        ngram (np.ndarray): The n-dimensional n-gram tensor. Shape = (V, V, ..., V)
        # dict_tokens (dict): Mapping from token -> index.
        # dict_in (dict): Mapping from index -> token.
        vocab (list of str): Subword tokens used by the tokenizer.

    Returns:
        str: The generated text with underscores replaced by spaces.
    """
    # n = ngram.ndim  # order of the n-gram model
    ngram = ngrams[n-1] # ensure it's a NumPy array (redundant if it already is)

    # define end_tokens as punctuation
    end_tokens = ['.', ':', '?', '!']

    # Tokenize input context
    text = to_byte_pair(context, vocab)

    next_word = ''
    while next_word not in end_tokens and len(text) <= 200:
        # Get the last (n-1) or n tokens for context (depending on your convention)
        # print(text)
        print("längeeeee", len(text))
        context = text[-(n-1):]

        preceding_keys = tuple(context)

        def backoff(n, ngrams, keys):
            print(keys)
            if keys in ngrams[n-1].keys():
                
                sorted_dict = {key: value for key, value in sorted(ngrams[n-1][keys].items(), key=lambda item: item[1], reverse=True)}
                # print(sorted_dict)
                zufall = random.randint(0,(len(sorted_dict)-1)//2)
                next_token = list(sorted_dict.keys())[zufall]

                return next_token
            else:
                print("wrong!!!")
                if n == 1:
                    return 'my_'
                return backoff(n-1, ngrams, keys)


        next_word = backoff(n, ngrams, preceding_keys)
        # Map tokens to indices; if unknown, use None
        # indices = [dict_tokens.get(token) for token in context]
        #
        # if None in indices:
        #     # If unknown tokens: fallback to overall next-token distribution
        #     column = ngram.sum(axis=tuple(range(n - 1)))
        #     next_idx = column.argmax()
        # else:
        #     # Build index tuple: fix context, free last axis
        #     index = tuple(indices) + (slice(None),)
        #
        #     # Get conditional next-token distribution
        #     column = ngram[index]
        #
        #     next_idx = column.argmax()

        # next_word = dict_in[next_idx]
        # text.append(next_word)
        
        text.append(next_word)
        # Join final tokens and clean up
        # pretty_text = ''.join(text)
        # pretty_text = pretty_text.replace('_', ' ')
        #print(pretty_text)
        # print(text)
    # text = str(text)
    # Join final tokens and clean up
    pretty_text = "".join(str(x) for x in text)
    # pretty_text = ''.join(text)
    pretty_text = pretty_text.replace('_', ' ')
    return pretty_text

In [130]:
#from generator import generate
n_list = [my_unigram.n_gram_probs, my_bigram.n_gram_probs]

text = generate("cleopatra is my", n_list, 2, vocab_train)
print(text)


längeeeee 3
('my_',)
wrong!!!
('my_',)
wrong!!!
längeeeee 4
('my_',)
wrong!!!
('my_',)
wrong!!!
längeeeee 5
('my_',)
wrong!!!
('my_',)
wrong!!!
längeeeee 6
('my_',)
wrong!!!
('my_',)
wrong!!!
längeeeee 7
('my_',)
wrong!!!
('my_',)
wrong!!!
längeeeee 8
('my_',)
wrong!!!
('my_',)
wrong!!!
längeeeee 9
('my_',)
wrong!!!
('my_',)
wrong!!!
längeeeee 10
('my_',)
wrong!!!
('my_',)
wrong!!!
längeeeee 11
('my_',)
wrong!!!
('my_',)
wrong!!!
längeeeee 12
('my_',)
wrong!!!
('my_',)
wrong!!!
längeeeee 13
('my_',)
wrong!!!
('my_',)
wrong!!!
längeeeee 14
('my_',)
wrong!!!
('my_',)
wrong!!!
längeeeee 15
('my_',)
wrong!!!
('my_',)
wrong!!!
längeeeee 16
('my_',)
wrong!!!
('my_',)
wrong!!!
längeeeee 17
('my_',)
wrong!!!
('my_',)
wrong!!!
längeeeee 18
('my_',)
wrong!!!
('my_',)
wrong!!!
längeeeee 19
('my_',)
wrong!!!
('my_',)
wrong!!!
längeeeee 20
('my_',)
wrong!!!
('my_',)
wrong!!!
längeeeee 21
('my_',)
wrong!!!
('my_',)
wrong!!!
längeeeee 22
('my_',)
wrong!!!
('my_',)
wrong!!!
längeeeee 23
('my_',)
wrong

In [ ]:
my_to_byte = to_byte_pair("hello, i am shakespeare, hello", vocab_train[100:])
print(my_to_byte)
my_own_gram = N_gram(my_to_byte, 3)

print(f"Unigram probabilities of test trigram: {my_own_gram.unigram_probs}")
print(f"Chunking of test trigram: {my_own_gram.split_text}")
my_own_trigram_probs = my_own_gram.calculate_n_gram_probs(my_own_gram.split_text, 3)
print(f"Trigram probabilities: {my_own_trigram_probs}")

['hel', 'lo', ',_', 'i_', 'am_', 'sha', 'ke', 'spea', 're', ',_', 'hel', 'lo']
Unigram probabilities of test trigram: {'hel': 0.16666666666666666, 'lo': 0.16666666666666666, ',_': 0.16666666666666666, 'i_': 0.08333333333333333, 'am_': 0.08333333333333333, 'sha': 0.08333333333333333, 'ke': 0.08333333333333333, 'spea': 0.08333333333333333, 're': 0.08333333333333333}
Chunking of test trigram: [['hel', 'lo', ',_'], ['lo', ',_', 'i_'], [',_', 'i_', 'am_'], ['i_', 'am_', 'sha'], ['am_', 'sha', 'ke'], ['sha', 'ke', 'spea'], ['ke', 'spea', 're'], ['spea', 're', ',_'], ['re', ',_', 'hel'], [',_', 'hel', 'lo']]
Trigram probabilities: {('hel', 'lo'): {',_': 1.0}, ('lo', ',_'): {'i_': 1.0}, (',_', 'i_'): {'am_': 1.0}, ('i_', 'am_'): {'sha': 1.0}, ('am_', 'sha'): {'ke': 1.0}, ('sha', 'ke'): {'spea': 1.0}, ('ke', 'spea'): {'re': 1.0}, ('spea', 're'): {',_': 1.0}, ('re', ',_'): {'hel': 1.0}, (',_', 'hel'): {'lo': 1.0}}


In [ ]:
def get_top_bigrams(n_gram_probs, k=5):
    bigram_list = []
    for context, next_words in n_gram_probs.items():
        for word, prob in next_words.items():
            bigram = context + (word,)
            bigram_list.append((bigram, prob))

    # Sort by probability (descending)
    bigram_list.sort(key=lambda x: x[1], reverse=True)

    # Print top k
    print(f"Top {k} most probable bigrams:")
    for i in range(min(k, len(bigram_list))):
        bigram, prob = bigram_list[i]
        print(f"{bigram} → {prob:.4f}")

get_top_bigrams(my_bigram_probs, 20)

Top 20 most probable bigrams:
('\n\n\n\n', '_') → 1.0000
(':', '\n') → 1.0000
('\n\n\n', '_') → 1.0000
(';', '\n') → 1.0000
('?', '\n') → 1.0000
('dies_', 'with_') → 1.0000
('cy', '?') → 1.0000
('gon', 'e') → 1.0000
('exe', 'cu') → 1.0000
('lee', 'se_') → 1.0000
('shou', 'ld') → 1.0000
('maj', 'est') → 1.0000
('ore_', 'his_') → 1.0000
('tch_', 'with_') → 1.0000
('d-', 'hear') → 1.0000
('cold_', 'de') → 1.0000
('prin', 't_') → 1.0000
('viol', 'et_') → 1.0000
('hen', 'ce') → 1.0000
('house_', 'fall_') → 1.0000
